#   Captcha Training Model  And we got the accuracy of above 92 %

## This is 6 letter captcha for prediction 
## i am not providing any data set or any captcha due to some security reason.

In [1]:
# Some important library
import glob
import time
import os
import re
import shutil
import selenium
import urllib.request
import pytesseract
import requests
import datetime
from datetime import timedelta
from datetime import date
from tqdm import tqdm
from PIL import Image
from PIL import ImageFilter
import cv2
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt # graphics


def prepare_image(img):
    """Transform image to greyscale and blur it"""
    img = img.filter(ImageFilter.SMOOTH_MORE)
    img = img.filter(ImageFilter.SMOOTH_MORE)
    if 'L' != img.mode:
        img = img.convert('L')
    return img

def remove_noise(img, pass_factor):
    for column in range(img.size[0]):
        for line in range(img.size[1]):
            value = remove_noise_by_pixel(img, column, line, pass_factor)
            img.putpixel((column, line), value)
    return img

def remove_noise_by_pixel(img, column, line, pass_factor):
    if img.getpixel((column, line)) < pass_factor:
        return (0)
    return (255)
pass_factor=186

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
#  we can also use glob to get all files and easy to get using glob
filelist = []

import os
for dirname, _, filenames in os.walk(r'C:\Users\ADMIN\Desktop\Python\Code\ML\Captcha Model\Captcha'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        filelist.append(os.path.join(dirname, filename))

In [5]:
filelist[:1]

['C:\\Users\\ADMIN\\Desktop\\Python\\Code\\ML\\Captcha Model\\Captcha\\009r1z.png']

In [20]:
# creating two empty list for appending images and labels data
labels = []
images = []
#Importing Images from File-List
for image_path in filelist[:1500]:
    image = Image.open(image_path).filter(ImageFilter.SHARPEN)
    image = image.convert('L') #convert image to black and white
    pass_factor = 186
    image = prepare_image(image) # applying function for preparing image
    image=remove_noise(image,pass_factor) 
    image = np.array(image)  # convert image to array
    image = image/255 #Normalization
    
    image_label = image_path.split("\\")[-1].split(".")[0]
    
    images.append(image)
    labels.append(image_label)

In [21]:
#Stacking Images to a Numpy Array
images = np.stack(images)
print(images.shape)

(1500, 80, 200)


In [22]:
#One-Hot Encoding List of 6 character strings
def one_hot_encode(characters, char_pool):
    char_to_index = {char: i for i, char in enumerate(char_pool)}
    num_classes = len(char_pool)
    encoding_size = len(characters[0])
    
    one_hot_encoded = np.zeros((len(characters), encoding_size, num_classes), dtype=int)

    for i, word in enumerate(characters):
        for j, char in enumerate(word):
            index = char_to_index[char]
            one_hot_encoded[i, j, index] = 1

    return one_hot_encoded

# Character pool: digits 0-9 and lowercase letters a-z
char_pool = '0123456789abcdefghijklmnopqrstuvwxyz'


In [23]:
#Encoding Labels
encoded_labels = one_hot_encode(labels, char_pool)
encoded_labels = np.array(encoded_labels)
# encoded_labels.shape, encoded_labels

In [24]:
#Shuffling Array
indices = np.random.permutation(len(labels))

images = images[indices]
encoded_labels = encoded_labels[indices]

images.shape, encoded_labels.shape

((1500, 80, 200), (1500, 6, 36))

In [43]:
# We can also use this for split
# from sklearn.model_selection import train_test_split
# X = encoded_labels
# y = images
# train_labels, val_labels, train_images, val_images = train_test_split(X,y,test_size=0.1,random_state=42)

In [46]:
#Train-Validation Split
split_index = int(len(labels) * 0.9)

train_labels = encoded_labels[:split_index]
val_labels = encoded_labels[split_index:]

train_images = images[:split_index]
val_images = images[split_index:]

In [47]:
val_images.shape

(150, 80, 200)

In [74]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [75]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Activation, BatchNormalization, MaxPooling2D, Dropout, Flatten, Dense, Reshape

model = Sequential([
    Conv2D(32, (3, 3), input_shape=(80, 200, 1)),  # conv2d_6
    Activation('relu'),                            # activation_9
    BatchNormalization(),                          # batch_normalization_7
    MaxPooling2D((2, 2)),                          # max_pooling2d_4
    Dropout(0.25),                                 # dropout_6
    
    Conv2D(32, (3, 3)),                            # conv2d_7
    Activation('relu'),                            # activation_10
    BatchNormalization(),                          # batch_normalization_8
    Conv2D(32, (3, 3)),                            # conv2d_8
    Activation('relu'),                            # activation_11
    BatchNormalization(),                          # batch_normalization_9
    MaxPooling2D((2, 2)),                          # max_pooling2d_5
    Dropout(0.25),                                 # dropout_7
    
    Conv2D(64, (3, 3)),                            # conv2d_9
    Activation('relu'),                            # activation_12
    BatchNormalization(),                          # batch_normalization_10
    MaxPooling2D((2, 2)),                          # max_pooling2d_6
    Dropout(0.25),                                 # dropout_8
    
    Conv2D(64, (1, 1)),                            # conv2d_10 (changed kernel size to (1, 1))
    Activation('relu'),                            # activation_13
    BatchNormalization(),                          # batch_normalization_11
    MaxPooling2D((2, 2)),                          # max_pooling2d_7
    Dropout(0.25),                                 # dropout_9
    
    Conv2D(128, (1, 1)),                           # conv2d_11 (changed kernel size to (1, 1))
    Activation('relu'),                            # activation_14
    Dropout(0.25),                                 # dropout_10
    
    Flatten(),                                     # flatten_1
    Dense(2048),                                   # dense_4
    Activation('relu'),                            # activation_15
    BatchNormalization(),                          # batch_normalization_12
    Dropout(0.5),                                  # dropout_10
    
    Dense(1024),                                   # dense_5
    Activation('relu'),                            # activation_16
    BatchNormalization(),                          # batch_normalization_13
    Dropout(0.5),                                  # dropout_11
    
    Dense(216),                                    # dense_6
    Reshape((6, 36)),                              # reshape_1
    Dense(36),                                     # dense_7
    Activation('softmax')                          # activation_17
])

model.summary()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 78, 198, 32)       320       
                                                                 
 activation (Activation)     (None, 78, 198, 32)       0         
                                                                 
 batch_normalization (Batch  (None, 78, 198, 32)       128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 39, 99, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 39, 99, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 37, 97, 32)        9

In [77]:
history = model.fit(train_images, train_labels, epochs=50, 
                    validation_data=(val_images, val_labels))

Epoch 1/50


43/43 [==============================] - 25s 418ms/step - loss: 4.6993 - accuracy: 0.0354 - val_loss: 4.0830 - val_accuracy: 0.0222
Epoch 2/50
43/43 [==============================] - 17s 387ms/step - loss: 3.8814 - accuracy: 0.0878 - val_loss: 6.2740 - val_accuracy: 0.0289
Epoch 3/50
43/43 [==============================] - 17s 386ms/step - loss: 3.0629 - accuracy: 0.1653 - val_loss: 9.2854 - val_accuracy: 0.0333
Epoch 4/50
43/43 [==============================] - 16s 383ms/step - loss: 2.4198 - accuracy: 0.2647 - val_loss: 11.0711 - val_accuracy: 0.0322
Epoch 5/50
43/43 [==============================] - 17s 384ms/step - loss: 2.0242 - accuracy: 0.3426 - val_loss: 11.8784 - val_accuracy: 0.0311
Epoch 6/50
43/43 [==============================] - 16s 383ms/step - loss: 1.7512 - accuracy: 0.4191 - val_loss: 13.0552 - val_accuracy: 0.0233
Epoch 7/50
43/43 [==============================] - 17s 385ms/step - loss: 1.5341 - accuracy: 0.4705 - val_loss: 13.2906 - val_accuracy: 

In [81]:
# Save the model
model.save(r'C:\Users\DELL\Desktop\Python\Code\Capture\Captcha_model3.1.h5')

In [1]:
# This is for prediction of captcha using trained model
import numpy as np
from PIL import Image,ImageFilter
from tensorflow.keras.models import load_model
pass_factor = 179

def prepare_image(img):
    """Transform image to greyscale and blur it"""
    img = img.filter(ImageFilter.SMOOTH_MORE)
    img = img.filter(ImageFilter.SMOOTH_MORE)
    if 'L' != img.mode:
        img = img.convert('L')
    return img

def remove_noise_by_pixel(img, column, line, pass_factor):
    if img.getpixel((column, line)) < pass_factor:
        return (0)
    return (255)

def remove_noise(img, pass_factor):
    for column in range(img.size[0]):
        for line in range(img.size[1]):
            value = remove_noise_by_pixel(img, column, line, pass_factor)
            img.putpixel((column, line), value)
    return img


def preprocess_image(image_path):
    # Load the image and preprocess it
    image = Image.open(image_path).filter(ImageFilter.SHARPEN).convert("L")  # Convert to grayscale
    image = prepare_image(image)
    image = remove_noise(image,pass_factor)
    image = np.array(image) / 255.0  # Normalize pixel values to [0, 1]
    image = np.expand_dims(image, axis=-1)  # Add a channel dimension
    image = np.expand_dims(image, axis=0)  # Add a batch dimension
    return image

def decode_captcha(predicted_captcha):
    # Implement your logic to decode the predicted captcha
    # This depends on how the labels were encoded during training
    # You might need to reverse the one-hot encoding and map it to characters
    # For example, if labels were one-hot encoded using characters '0123456789abcdefghijklmnopqrstuvwxyz',
    # you can reverse the one-hot encoding and map it back to characters using argmax function
    
    # Placeholder logic, modify according to your label encoding
    characters = '0123456789abcdefghijklmnopqrstuvwxyz'
    decoded_captcha = ''
    for prediction in predicted_captcha[0]:
        index = np.argmax(prediction)
        decoded_captcha += characters[index]
    
    return decoded_captcha

def predict_captcha(image_path, model):
    # Preprocess the image
    image = preprocess_image(image_path)
    
    # Load the trained model
#     model = load_model(model_path)
    
    # Predict the captcha
    predicted_captcha = model.predict(image)
    
    # Convert the predicted captcha from one-hot encoding to a string
    captcha_string = decode_captcha(predicted_captcha)
    
    return captcha_string


# Example usage:
image_path = r'C:\Users\DELL\Desktop\Python\Code\Capture\iwum4g.png'  # Replace with the path to your captcha image
model_path = r'C:\Users\DELL\Desktop\Python\Captcha_model3.3.h5'   # Replace with the path to your trained model
model = load_model(model_path)
predicted_captcha = predict_captcha(image_path, model)
print("Predicted Captcha:", predicted_captcha)




1/1 [==============================] - 0s 353ms/step
Predicted Captcha: fwam4g
